# 1. Set Default Value for Variable

---

```js
const num = (req.query.number === undefined) ? 20 : req.query.number;
const date = (req.query.date === undefined) ? new Date() : req.query.date;

// Limit the number of results, Temperature is Mongoose Model
const results = await Temperature.find({limit: 20});
res.status(200).json(results);
```


# 2. Time

---

## 2.1. Convert Date to yyyy-mm-dd

You can use this store date string to database.

```js
console.log(new Date());
// 2022-09-23T14:35:39.074Z

const todayDate = new Date().toISOString().slice(0, 10);
console.log(todayDate);
// 2022-09-23
```

## 2.2. Query with Date


```js

const getTemperature = async (req, res) => {
    const { startDate, endDate } = req.query;

    if(startDate === '' || endDate === '') {
        return res.status(400).json({
            status:'failure',
            message: 'You must specify \'starDate\' and \'endDate\'.'
        })
    }

    let results;
    try {
        results = await Temperature.find({
            create_date: {
                $gte: new Date(new Date(startDate).setHours(0, 0, 0)),
                $lt: new Date(new Date(endDate).setHours(23, 59, 59))
            }
        }).sort({ date: 'asc'});
    } catch (error) {
        res.status(404).json({ message: error.message });
    }

    if(results.length === 0) {
        return res.status(404).json({
            status:'failure',
            message:`Could not retrieve data from ${startDate} to ${endDate}`
        })
    }

    res.status(200).json({
        status:'success',
        data: results
    });
}
```

### curl 测试

```shell
http://localhost:3000/temperature/?endDate=2022-10-20&startDate=2022-09-01
```


## 2.3. Insert Data with Date

Use `new Date()` when inserting into MongoDB database. Do not use `new Date().toISOString()` when inserting into MongoDB database because MongoDB already does the conversion for you and using `.toISOString()` will make `.find()` operators or `$gte` operators on the database fail.

```js
const createTemperature = async (req, res) => {
    // when use req.body, you have to write app.use(cors()); in app.js first
    const value = req.body.value;
    if(value === undefined) {
        return res.status(400).json({
            status:'failure',
            message: 'You must specify value.'
        })
    }

    const create_date = new Date();
    const temperature = new Temperature({ value, create_date });
    try {
        await temperature.save();
        res.status(201).json(temperature);
    } catch (error) {
        res.status(409).json({ message: error.message });
    }
}
```


https://stackoverflow.com/a/61225732/16317008


### curl测试:

```shell
curl -d '{ "value": 50.6 }' -H "Content-Type: application/json" -X POST http://localhost:3000/temperature/
```

